In [94]:
import sys
import spotipy
import spotipy.util as util
import pandas as pd
import json
import os
import pandas
from pandas import read_csv
from user import spotify
import pickle
import re

In [95]:
user_name = "Pillsbury Darboy"
#client_id = '1e05e910ef664f27949005267fdd0936'
#client_secret ='7e65fb89a86a4a9c849924f5bd95661a'
client_id = "bff1afed3b63465aa6cdba9ec40f6afd"
client_secret = "53634fc86e8a4f71b7459197084e3cdf"
#client_id = 'c6bde56f528545d2a88a6ab3755e7c09'
#client_secret = '8d5ddcd8fed7421491bb1c2263879ced'
redirect_uri = "http://localhost/"
scope = 'user-library-read'
from spotipy.util import prompt_for_user_token
token = prompt_for_user_token(username=user_name,scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
from spotipy import Spotify
spotify = Spotify(auth=token)

In [ ]:
#pkl_file = open('mixes_th.pckl', 'rb')
#mixes_th = pickle.load(pkl_file)
#pkl_file = open('discogs_at.pckl', 'rb')
#discogs_th = pickle.load(pkl_file)
#pkl_file = open('discogs_filtered_techno_house.pckl', 'rb')
#discogs_filtered = pickle.load(pkl_file)

In [53]:
def spotify_parse_tracks(artist, track):
    global search_queries
    search_query = artist
    tracklist_frames = []
    if search_query in search_queries:
        print("Already searched %s" % search_query)
        return(tracklist_frames)
    else:
        result = spotify.search(q=search_query, limit=50)

    print('Parsing %s tracks' % result['tracks']['total'])
    if result['tracks']['total'] > 800:
        print('Too many tracks, narrowing query')
        search_query = str(artist) + ' ' + str(track)
        if search_query in search_queries:
            print("Already searched %s" % search_query)
            return(tracklist_frames)
        else:
            result = spotify.search(q=search_query, limit=50)
        if result['tracks']['total'] > 1000:
            print("%s returning too many results" % search_query)
            search_queries.append(search_query)
            return(tracklist_frames)
        print('Parsing %s tracks' % result['tracks']['total'])
        
    searching = True
    search_queries.append(search_query)
    i = 1
    while searching:
        tracks_raw = result['tracks']['items']
        for track in tracks_raw:
            artists = []
            for artist in track['artists']:
                artists.append(artist['name'])

            tracklist_frames.append({'track': track['name'],'artist': artists[0], 'artists': artists, 'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']})
        print("Parsed page %s" % i)
        i+=1
        if result['tracks']['next']:
            result = spotify.next(result['tracks'])
        else:
            searching = False
    return(tracklist_frames)
    

In [54]:
def spotify_parse_audio_features(id_list):
    length = len(id_list)
    feature_frame = []
    x = 0
    y = 49
    while y <= length:
        if length < y:
            y = length
        results = spotify.audio_features(tracks=id_list[x:y])
        if None in results:
            results.remove(None)
        try:
            feature_frame.append(pd.DataFrame(results))
        except: 
            print("Error appending dataframe")
        print('Parsing pages %s to %s' % (x,y))
        x += 50
        y += 50
    return(feature_frame)

In [55]:
def clean_artist_name(artist):
    return(re.sub(pattern='\([0-9]+\)', string = artist, repl='').strip())

In [ ]:
#result_frame = []
for i in range(71174, len(discogs_filtered)):
    artist = clean_artist_name(discogs_filtered['artist'][i])
    track = discogs_filtered['track'][i]
    result_frame.append(pd.DataFrame(spotify_parse_tracks(artist,track)))
    if i % 5000 == 0:
        results = pd.concat(result_frame)
        results = results.reset_index(drop=True)
        nodup = results[['artist', 'track']].drop_duplicates()
        #search_queriesbackup = search_queries
        search_queries.extend(nodup['artist'] + ' ' + nodup['track'])
        search_queries = list(set(search_queries))
        print('--------------------')
        print('--------------------')
        print("RESHAPING DATA FRAME")
        print('--------------------')
        print('--------------------')
        user_name = "JohnaaDoe"
        #client_id = '1e05e910ef664f27949005267fdd0936'
        #client_secret ='7e65fb89a86a4a9c849924f5bd95661a'
        client_id = "bff1afed3b63465aa6cdba9ec40f6afd"
        client_secret = "53634fc86e8a4f71b7459197084e3cdf"
        #client_id = 'c6bde56f528545d2a88a6ab3755e7c09'
        #client_secret = '8d5ddcd8fed7421491bb1c2263879ced'
        redirect_uri = "http://localhost/"
        scope = 'user-library-read'
        from spotipy.util import prompt_for_user_token
        token = prompt_for_user_token(username=user_name,scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
        from spotipy import Spotify
        spotify = Spotify(auth=token)
    #results = pd.concat(result_frame)

In [ ]:
#results = pd.concat(result_frame)
#results = results.reset_index(drop=True)
#nodup = results[['artist', 'track']].drop_duplicates()
#search_queriesbackup = search_queries
#earch_queries.extend(nodup['artist'] + ' ' + nodup['track'])
#search_queries = list(set(search_queries))
#import pickle
#fileHandle = open('spotify_database_track_ids_frame6.pckl', "wb")
#pickle.dump(results, fileHandle)
#fileHandle = open('search_queries_master.pckl', "wb")
#pickle.dump(search_queries, fileHandle)
#import pickle
#fileHandle = open('spotify_database_track_ids.pckl', "wb")
#pickle.dump(results, fileHandle)


# Read and Merge spotify track ids and audio features

In [104]:
df_master = pd.read_csv('data/spotify/spotify_database_track_ids_master.csv')
del df_master['Unnamed: 0']
df_master = df_master.rename(columns={'album':'release'})


In [4]:
df_audio_features = pd.read_csv('data/spotify/spotify_database_audio_features_master.csv')
del df_audio_features['Unnamed: 0']
del df_audio_features['0']
del df_audio_features['Unnamed: 0.1']
del df_audio_features['0.1']

/Users/hitchhiker/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
df_master[df_master['track'] == 'Evviva Palm Town']

,release,artist,disc_number,duration,id,popularity,track,track_number


In [40]:
z = spotify.tracks(tracks=['2ECnuKMDzS79VVOlhndEFT','3LFbUTCBW3Ie5M70ockasP'])

In [ ]:
missing_spotify = pd.read_csv('data/spotify/missing_spotify_audio_features.csv')

In [108]:
df_final = df_master.merge(df_audio_features, on='id')

In [109]:
len(df_final)

4650019

In [110]:
df_final.to_csv('data/spotify/spotify_database_mergedV2.csv')

In [107]:
df_master.to_csv('data/spotify/spotify_metadata_final.csv')

In [6]:
to_concat = [df_audio_features, missing_spotify]
df_audio_features = pd.concat(to_concat)

In [105]:
x = set(df_audio_features.id)
y = set(df_master.id)

In [106]:
# Items possessed by y not possessed by x
y.difference(x)
# Items possessed by x not possessed by y
len(x.difference(y))

21826

In [14]:
z = list(x.difference(y)) # missing metadata
z.pop(0)

In [21]:
spotify.tracks(z[0:49])

{'tracks': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3E1yxE05BnQblIGuJ7yadK'},
      'href': 'https://api.spotify.com/v1/artists/3E1yxE05BnQblIGuJ7yadK',
      'id': '3E1yxE05BnQblIGuJ7yadK',
      'name': 'Dream Baby',
      'type': 'artist',
      'uri': 'spotify:artist:3E1yxE05BnQblIGuJ7yadK'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/2W7bmTHguay2jxEAdw9Flc'},
      'href': 'https://api.spotify.com/v1/artists/2W7bmTHguay2jxEAdw9Flc',
      'id': '2W7bmTHguay2jxEAdw9Flc',
      'name': 'Sleep Music Lullabies',
      'type': 'artist',
      'uri': 'spotify:artist:2W7bmTHguay2jxEAdw9Flc'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/4wCm4MDz3IiWQtjRtMtVwf'},
      'href': 'https://api.spotify.com/v1/artists/4wCm4MDz3IiWQtjRtMtVwf',
      'id': '4wCm4MDz3IiWQtjRtMtVwf',
      'name': 'Bedtime for Baby',
      'type': 'artist',
      'uri': 'spotify:artist:4wCm4MDz3I

In [76]:
def spotify_parse_missing_ids(id_list):
    length = len(id_list)
    feature_frame = []
    x = 0
    y = 49
    while y <= length:
        if length < y:
            y = length
        results = spotify.tracks(tracks=id_list[x:y])
        if None in results:
            results.remove(None)
        for track in results['tracks']:
            try:
                feature_frame.append({'track': track['name'],'artist': track['artists'][0]['name'], 'duration': track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'release':track['album']['name']})
            except: 
                print("Error appending dataframe")
        print('Parsing pages %s to %s' % (x,y))
        x += 50
        y += 50
    return(pd.DataFrame(feature_frame))

In [79]:
test_1 = spotify_parse_missing_ids(z[0:10000])

Parsing pages 0 to 49
Parsing pages 50 to 99
Parsing pages 100 to 149
Parsing pages 150 to 199
Parsing pages 200 to 249
Parsing pages 250 to 299
Parsing pages 300 to 349
Parsing pages 350 to 399
Parsing pages 400 to 449
Parsing pages 450 to 499
Parsing pages 500 to 549
Parsing pages 550 to 599
Parsing pages 600 to 649
Parsing pages 650 to 699
Parsing pages 700 to 749
Parsing pages 750 to 799
Parsing pages 800 to 849
Parsing pages 850 to 899
Parsing pages 900 to 949
Parsing pages 950 to 999
Parsing pages 1000 to 1049
Parsing pages 1050 to 1099
Parsing pages 1100 to 1149
Parsing pages 1150 to 1199
Parsing pages 1200 to 1249
Parsing pages 1250 to 1299
Parsing pages 1300 to 1349
Parsing pages 1350 to 1399
Parsing pages 1400 to 1449
Parsing pages 1450 to 1499
Parsing pages 1500 to 1549
Parsing pages 1550 to 1599
Parsing pages 1600 to 1649
Parsing pages 1650 to 1699
Parsing pages 1700 to 1749
Parsing pages 1750 to 1799
Parsing pages 1800 to 1849
Parsing pages 1850 to 1899
Parsing pages 1900 

In [81]:
test_2 = spotify_parse_missing_ids(z[10000:100000])

Parsing pages 0 to 49
Parsing pages 50 to 99
Parsing pages 100 to 149
Parsing pages 150 to 199
Parsing pages 200 to 249
Parsing pages 250 to 299
Parsing pages 300 to 349
Parsing pages 350 to 399
Parsing pages 400 to 449
Parsing pages 450 to 499
Parsing pages 500 to 549
Parsing pages 550 to 599
Parsing pages 600 to 649
Parsing pages 650 to 699
Parsing pages 700 to 749
Parsing pages 750 to 799
Parsing pages 800 to 849
Parsing pages 850 to 899
Parsing pages 900 to 949
Parsing pages 950 to 999
Parsing pages 1000 to 1049
Parsing pages 1050 to 1099
Parsing pages 1100 to 1149
Parsing pages 1150 to 1199
Parsing pages 1200 to 1249
Parsing pages 1250 to 1299
Parsing pages 1300 to 1349
Parsing pages 1350 to 1399
Parsing pages 1400 to 1449
Parsing pages 1450 to 1499
Parsing pages 1500 to 1549
Parsing pages 1550 to 1599
Parsing pages 1600 to 1649
Parsing pages 1650 to 1699
Parsing pages 1700 to 1749
Parsing pages 1750 to 1799
Parsing pages 1800 to 1849
Parsing pages 1850 to 1899
Parsing pages 1900 

Parsing pages 14900 to 14949
Parsing pages 14950 to 14999
Parsing pages 15000 to 15049
Parsing pages 15050 to 15099
Parsing pages 15100 to 15149
Parsing pages 15150 to 15199
Parsing pages 15200 to 15249
Parsing pages 15250 to 15299
Parsing pages 15300 to 15349
Parsing pages 15350 to 15399
Parsing pages 15400 to 15449
Parsing pages 15450 to 15499
Parsing pages 15500 to 15549
Parsing pages 15550 to 15599
Parsing pages 15600 to 15649
Parsing pages 15650 to 15699
Parsing pages 15700 to 15749
Parsing pages 15750 to 15799
Parsing pages 15800 to 15849
Parsing pages 15850 to 15899
Parsing pages 15900 to 15949
Parsing pages 15950 to 15999
Parsing pages 16000 to 16049
Parsing pages 16050 to 16099
Parsing pages 16100 to 16149
Parsing pages 16150 to 16199
Parsing pages 16200 to 16249
Parsing pages 16250 to 16299
Parsing pages 16300 to 16349
Parsing pages 16350 to 16399
Parsing pages 16400 to 16449
Parsing pages 16450 to 16499
Parsing pages 16500 to 16549
Parsing pages 16550 to 16599
Parsing pages 

Parsing pages 29050 to 29099
Parsing pages 29100 to 29149
Parsing pages 29150 to 29199
Parsing pages 29200 to 29249
Parsing pages 29250 to 29299
Parsing pages 29300 to 29349
Parsing pages 29350 to 29399
Parsing pages 29400 to 29449
Parsing pages 29450 to 29499
Parsing pages 29500 to 29549
Parsing pages 29550 to 29599
Parsing pages 29600 to 29649
Parsing pages 29650 to 29699
Parsing pages 29700 to 29749
Parsing pages 29750 to 29799
Parsing pages 29800 to 29849
Parsing pages 29850 to 29899
Parsing pages 29900 to 29949
Parsing pages 29950 to 29999
Parsing pages 30000 to 30049
Parsing pages 30050 to 30099
Parsing pages 30100 to 30149
Parsing pages 30150 to 30199
Parsing pages 30200 to 30249
Parsing pages 30250 to 30299
Parsing pages 30300 to 30349
Parsing pages 30350 to 30399
Parsing pages 30400 to 30449
Parsing pages 30450 to 30499
Parsing pages 30500 to 30549
Parsing pages 30550 to 30599
Parsing pages 30600 to 30649
Parsing pages 30650 to 30699
Parsing pages 30700 to 30749
Parsing pages 

Parsing pages 43200 to 43249
Parsing pages 43250 to 43299
Parsing pages 43300 to 43349
Parsing pages 43350 to 43399
Parsing pages 43400 to 43449
Parsing pages 43450 to 43499
Parsing pages 43500 to 43549
Parsing pages 43550 to 43599
Parsing pages 43600 to 43649
Parsing pages 43650 to 43699
Parsing pages 43700 to 43749
Parsing pages 43750 to 43799
Parsing pages 43800 to 43849
Parsing pages 43850 to 43899
Parsing pages 43900 to 43949
Parsing pages 43950 to 43999
Parsing pages 44000 to 44049
Parsing pages 44050 to 44099
Parsing pages 44100 to 44149
Parsing pages 44150 to 44199
Parsing pages 44200 to 44249
Parsing pages 44250 to 44299
Parsing pages 44300 to 44349
Parsing pages 44350 to 44399
Parsing pages 44400 to 44449
Parsing pages 44450 to 44499
Parsing pages 44500 to 44549
Parsing pages 44550 to 44599
Parsing pages 44600 to 44649
Parsing pages 44650 to 44699
Parsing pages 44700 to 44749
Parsing pages 44750 to 44799
Parsing pages 44800 to 44849
Parsing pages 44850 to 44899
Parsing pages 

Parsing pages 57350 to 57399
Parsing pages 57400 to 57449
Parsing pages 57450 to 57499
Parsing pages 57500 to 57549
Parsing pages 57550 to 57599
Parsing pages 57600 to 57649
Parsing pages 57650 to 57699
Parsing pages 57700 to 57749
Parsing pages 57750 to 57799
Parsing pages 57800 to 57849
Parsing pages 57850 to 57899
Parsing pages 57900 to 57949
Parsing pages 57950 to 57999
Parsing pages 58000 to 58049
Parsing pages 58050 to 58099
Parsing pages 58100 to 58149
Parsing pages 58150 to 58199
Parsing pages 58200 to 58249
Parsing pages 58250 to 58299
Parsing pages 58300 to 58349
Parsing pages 58350 to 58399
Parsing pages 58400 to 58449
Parsing pages 58450 to 58499
Parsing pages 58500 to 58549
Parsing pages 58550 to 58599
Parsing pages 58600 to 58649
Parsing pages 58650 to 58699
Parsing pages 58700 to 58749
Parsing pages 58750 to 58799
Parsing pages 58800 to 58849
Parsing pages 58850 to 58899
Parsing pages 58900 to 58949
Parsing pages 58950 to 58999
Parsing pages 59000 to 59049
Parsing pages 

Parsing pages 71500 to 71549
Parsing pages 71550 to 71599
Parsing pages 71600 to 71649
Parsing pages 71650 to 71699
Parsing pages 71700 to 71749
Parsing pages 71750 to 71799
Parsing pages 71800 to 71849
Parsing pages 71850 to 71899
Parsing pages 71900 to 71949
Parsing pages 71950 to 71999
Parsing pages 72000 to 72049
Parsing pages 72050 to 72099
Parsing pages 72100 to 72149
Parsing pages 72150 to 72199
Parsing pages 72200 to 72249
Parsing pages 72250 to 72299
Parsing pages 72300 to 72349
Parsing pages 72350 to 72399
Parsing pages 72400 to 72449
Parsing pages 72450 to 72499
Parsing pages 72500 to 72549
Parsing pages 72550 to 72599
Parsing pages 72600 to 72649
Parsing pages 72650 to 72699
Parsing pages 72700 to 72749
Parsing pages 72750 to 72799
Parsing pages 72800 to 72849
Parsing pages 72850 to 72899
Parsing pages 72900 to 72949
Parsing pages 72950 to 72999
Parsing pages 73000 to 73049
Parsing pages 73050 to 73099
Parsing pages 73100 to 73149
Parsing pages 73150 to 73199
Parsing pages 

Parsing pages 85650 to 85699
Parsing pages 85700 to 85749
Parsing pages 85750 to 85799
Parsing pages 85800 to 85849
Parsing pages 85850 to 85899
Parsing pages 85900 to 85949
Parsing pages 85950 to 85999
Parsing pages 86000 to 86049
Parsing pages 86050 to 86099
Parsing pages 86100 to 86149
Parsing pages 86150 to 86199
Parsing pages 86200 to 86249
Parsing pages 86250 to 86299
Parsing pages 86300 to 86349
Parsing pages 86350 to 86399
Parsing pages 86400 to 86449
Parsing pages 86450 to 86499
Parsing pages 86500 to 86549
Parsing pages 86550 to 86599
Parsing pages 86600 to 86649
Parsing pages 86650 to 86699
Parsing pages 86700 to 86749
Parsing pages 86750 to 86799
Parsing pages 86800 to 86849
Parsing pages 86850 to 86899
Parsing pages 86900 to 86949
Parsing pages 86950 to 86999
Parsing pages 87000 to 87049
Parsing pages 87050 to 87099
Parsing pages 87100 to 87149
Parsing pages 87150 to 87199
Parsing pages 87200 to 87249
Parsing pages 87250 to 87299
Parsing pages 87300 to 87349
Parsing pages 

In [84]:
test_3 = spotify_parse_missing_ids(z[100000:300000])
test_4 = spotify_parse_missing_ids(z[300000:500000])


Parsing pages 0 to 49
Parsing pages 50 to 99
Parsing pages 100 to 149
Parsing pages 150 to 199
Parsing pages 200 to 249
Parsing pages 250 to 299
Parsing pages 300 to 349
Parsing pages 350 to 399
Parsing pages 400 to 449
Parsing pages 450 to 499
Parsing pages 500 to 549
Parsing pages 550 to 599
Parsing pages 600 to 649
Parsing pages 650 to 699
Parsing pages 700 to 749
Parsing pages 750 to 799
Parsing pages 800 to 849
Parsing pages 850 to 899
Parsing pages 900 to 949
Parsing pages 950 to 999
Parsing pages 1000 to 1049
Parsing pages 1050 to 1099
Parsing pages 1100 to 1149
Parsing pages 1150 to 1199
Parsing pages 1200 to 1249
Parsing pages 1250 to 1299
Parsing pages 1300 to 1349
Parsing pages 1350 to 1399
Parsing pages 1400 to 1449
Parsing pages 1450 to 1499
Parsing pages 1500 to 1549
Parsing pages 1550 to 1599
Parsing pages 1600 to 1649
Parsing pages 1650 to 1699
Parsing pages 1700 to 1749
Parsing pages 1750 to 1799
Parsing pages 1800 to 1849
Parsing pages 1850 to 1899
Parsing pages 1900 

Parsing pages 14900 to 14949
Parsing pages 14950 to 14999
Parsing pages 15000 to 15049
Parsing pages 15050 to 15099
Parsing pages 15100 to 15149
Parsing pages 15150 to 15199
Parsing pages 15200 to 15249
Parsing pages 15250 to 15299
Parsing pages 15300 to 15349
Parsing pages 15350 to 15399
Parsing pages 15400 to 15449
Parsing pages 15450 to 15499
Parsing pages 15500 to 15549
Parsing pages 15550 to 15599
Parsing pages 15600 to 15649
Parsing pages 15650 to 15699
Parsing pages 15700 to 15749
Parsing pages 15750 to 15799
Parsing pages 15800 to 15849
Parsing pages 15850 to 15899
Parsing pages 15900 to 15949
Parsing pages 15950 to 15999
Parsing pages 16000 to 16049
Parsing pages 16050 to 16099
Parsing pages 16100 to 16149
Parsing pages 16150 to 16199
Parsing pages 16200 to 16249
Parsing pages 16250 to 16299
Parsing pages 16300 to 16349
Parsing pages 16350 to 16399
Parsing pages 16400 to 16449
Parsing pages 16450 to 16499
Parsing pages 16500 to 16549
Parsing pages 16550 to 16599
Parsing pages 

Parsing pages 29050 to 29099
Parsing pages 29100 to 29149
Parsing pages 29150 to 29199
Parsing pages 29200 to 29249
Parsing pages 29250 to 29299
Parsing pages 29300 to 29349
Parsing pages 29350 to 29399
Parsing pages 29400 to 29449
Parsing pages 29450 to 29499
Parsing pages 29500 to 29549
Parsing pages 29550 to 29599
Parsing pages 29600 to 29649
Parsing pages 29650 to 29699
Parsing pages 29700 to 29749
Parsing pages 29750 to 29799
Parsing pages 29800 to 29849
Parsing pages 29850 to 29899
Parsing pages 29900 to 29949
Parsing pages 29950 to 29999
Parsing pages 30000 to 30049
Parsing pages 30050 to 30099
Parsing pages 30100 to 30149
Parsing pages 30150 to 30199
Parsing pages 30200 to 30249
Parsing pages 30250 to 30299
Parsing pages 30300 to 30349
Parsing pages 30350 to 30399
Parsing pages 30400 to 30449
Parsing pages 30450 to 30499
Parsing pages 30500 to 30549
Parsing pages 30550 to 30599
Parsing pages 30600 to 30649
Parsing pages 30650 to 30699
Parsing pages 30700 to 30749
Parsing pages 

Parsing pages 43200 to 43249
Parsing pages 43250 to 43299
Parsing pages 43300 to 43349
Parsing pages 43350 to 43399
Parsing pages 43400 to 43449
Parsing pages 43450 to 43499
Parsing pages 43500 to 43549
Parsing pages 43550 to 43599
Parsing pages 43600 to 43649
Parsing pages 43650 to 43699
Parsing pages 43700 to 43749
Parsing pages 43750 to 43799
Parsing pages 43800 to 43849
Parsing pages 43850 to 43899
Parsing pages 43900 to 43949
Parsing pages 43950 to 43999
Parsing pages 44000 to 44049
Parsing pages 44050 to 44099
Parsing pages 44100 to 44149
Parsing pages 44150 to 44199
Parsing pages 44200 to 44249
Parsing pages 44250 to 44299
Parsing pages 44300 to 44349
Parsing pages 44350 to 44399
Parsing pages 44400 to 44449
Parsing pages 44450 to 44499
Parsing pages 44500 to 44549
Parsing pages 44550 to 44599
Parsing pages 44600 to 44649
Parsing pages 44650 to 44699
Parsing pages 44700 to 44749
Parsing pages 44750 to 44799
Parsing pages 44800 to 44849
Parsing pages 44850 to 44899
Parsing pages 

Parsing pages 57350 to 57399
Parsing pages 57400 to 57449
Parsing pages 57450 to 57499
Parsing pages 57500 to 57549
Parsing pages 57550 to 57599
Parsing pages 57600 to 57649
Parsing pages 57650 to 57699
Parsing pages 57700 to 57749
Parsing pages 57750 to 57799
Parsing pages 57800 to 57849
Parsing pages 57850 to 57899
Parsing pages 57900 to 57949
Parsing pages 57950 to 57999
Parsing pages 58000 to 58049
Parsing pages 58050 to 58099
Parsing pages 58100 to 58149
Parsing pages 58150 to 58199
Parsing pages 58200 to 58249
Parsing pages 58250 to 58299
Parsing pages 58300 to 58349
Parsing pages 58350 to 58399
Parsing pages 58400 to 58449
Parsing pages 58450 to 58499
Parsing pages 58500 to 58549
Parsing pages 58550 to 58599
Parsing pages 58600 to 58649
Parsing pages 58650 to 58699
Parsing pages 58700 to 58749
Parsing pages 58750 to 58799
Parsing pages 58800 to 58849
Parsing pages 58850 to 58899
Parsing pages 58900 to 58949
Parsing pages 58950 to 58999
Parsing pages 59000 to 59049
Parsing pages 

...

Parsing pages 145350 to 145399
Parsing pages 145400 to 145449
Parsing pages 145450 to 145499
Parsing pages 145500 to 145549
Parsing pages 145550 to 145599
Parsing pages 145600 to 145649
Parsing pages 145650 to 145699
Parsing pages 145700 to 145749
Parsing pages 145750 to 145799
Parsing pages 145800 to 145849
Parsing pages 145850 to 145899
Parsing pages 145900 to 145949
Parsing pages 145950 to 145999
Parsing pages 146000 to 146049
Parsing pages 146050 to 146099
Parsing pages 146100 to 146149
Parsing pages 146150 to 146199
Parsing pages 146200 to 146249
Parsing pages 146250 to 146299
Parsing pages 146300 to 146349
Parsing pages 146350 to 146399
Parsing pages 146400 to 146449
Parsing pages 146450 to 146499
Parsing pages 146500 to 146549
Parsing pages 146550 to 146599
Parsing pages 146600 to 146649
Parsing pages 146650 to 146699
Parsing pages 146700 to 146749
Parsing pages 146750 to 146799
Parsing pages 146800 to 146849
Parsing pages 146850 to 146899
Parsing pages 146900 to 146949
Parsing 

Parsing pages 158600 to 158649
Parsing pages 158650 to 158699
Parsing pages 158700 to 158749
Parsing pages 158750 to 158799
Parsing pages 158800 to 158849
Parsing pages 158850 to 158899
Parsing pages 158900 to 158949
Parsing pages 158950 to 158999
Parsing pages 159000 to 159049
Parsing pages 159050 to 159099
Parsing pages 159100 to 159149
Parsing pages 159150 to 159199
Parsing pages 159200 to 159249
Parsing pages 159250 to 159299
Parsing pages 159300 to 159349
Parsing pages 159350 to 159399
Parsing pages 159400 to 159449
Parsing pages 159450 to 159499
Parsing pages 159500 to 159549
Parsing pages 159550 to 159599
Parsing pages 159600 to 159649
Parsing pages 159650 to 159699
Parsing pages 159700 to 159749
Parsing pages 159750 to 159799
Parsing pages 159800 to 159849
Parsing pages 159850 to 159899
Parsing pages 159900 to 159949
Parsing pages 159950 to 159999
Parsing pages 160000 to 160049
Parsing pages 160050 to 160099
Parsing pages 160100 to 160149
Parsing pages 160150 to 160199
Parsing 

Parsing pages 171850 to 171899
Parsing pages 171900 to 171949
Parsing pages 171950 to 171999
Parsing pages 172000 to 172049
Parsing pages 172050 to 172099
Parsing pages 172100 to 172149
Parsing pages 172150 to 172199
Parsing pages 172200 to 172249
Parsing pages 172250 to 172299
Parsing pages 172300 to 172349
Parsing pages 172350 to 172399
Parsing pages 172400 to 172449
Parsing pages 172450 to 172499
Parsing pages 172500 to 172549
Parsing pages 172550 to 172599
Parsing pages 172600 to 172649
Parsing pages 172650 to 172699
Parsing pages 172700 to 172749
Parsing pages 172750 to 172799
Parsing pages 172800 to 172849
Parsing pages 172850 to 172899
Parsing pages 172900 to 172949
Parsing pages 172950 to 172999
Parsing pages 173000 to 173049
Parsing pages 173050 to 173099
Parsing pages 173100 to 173149
Parsing pages 173150 to 173199
Parsing pages 173200 to 173249
Parsing pages 173250 to 173299
Parsing pages 173300 to 173349
Parsing pages 173350 to 173399
Parsing pages 173400 to 173449
Parsing 

Parsing pages 185100 to 185149
Parsing pages 185150 to 185199
Parsing pages 185200 to 185249
Parsing pages 185250 to 185299
Parsing pages 185300 to 185349
Parsing pages 185350 to 185399
Parsing pages 185400 to 185449
Parsing pages 185450 to 185499
Parsing pages 185500 to 185549
Parsing pages 185550 to 185599
Parsing pages 185600 to 185649
Parsing pages 185650 to 185699
Parsing pages 185700 to 185749
Parsing pages 185750 to 185799
Parsing pages 185800 to 185849
Parsing pages 185850 to 185899
Parsing pages 185900 to 185949
Parsing pages 185950 to 185999
Parsing pages 186000 to 186049
Parsing pages 186050 to 186099
Parsing pages 186100 to 186149
Parsing pages 186150 to 186199
Parsing pages 186200 to 186249
Parsing pages 186250 to 186299
Parsing pages 186300 to 186349
Parsing pages 186350 to 186399
Parsing pages 186400 to 186449
Parsing pages 186450 to 186499
Parsing pages 186500 to 186549
Parsing pages 186550 to 186599
Parsing pages 186600 to 186649
Parsing pages 186650 to 186699
Parsing 

Parsing pages 198350 to 198399
Parsing pages 198400 to 198449
Parsing pages 198450 to 198499
Parsing pages 198500 to 198549
Parsing pages 198550 to 198599
Parsing pages 198600 to 198649
Parsing pages 198650 to 198699
Parsing pages 198700 to 198749
Parsing pages 198750 to 198799
Parsing pages 198800 to 198849
Parsing pages 198850 to 198899
Parsing pages 198900 to 198949
Parsing pages 198950 to 198999
Parsing pages 199000 to 199049
Parsing pages 199050 to 199099
Parsing pages 199100 to 199149
Parsing pages 199150 to 199199
Parsing pages 199200 to 199249
Parsing pages 199250 to 199299
Parsing pages 199300 to 199349
Parsing pages 199350 to 199399
Parsing pages 199400 to 199449
Parsing pages 199450 to 199499
Parsing pages 199500 to 199549
Parsing pages 199550 to 199599
Parsing pages 199600 to 199649
Parsing pages 199650 to 199699
Parsing pages 199700 to 199749
Parsing pages 199750 to 199799
Parsing pages 199800 to 199849
Parsing pages 199850 to 199899
Parsing pages 199900 to 199949
Parsing 

In [86]:
test_5 = spotify_parse_missing_ids(z[500000:700000])
test_6 = spotify_parse_missing_ids(z[700000:900000])


Parsing pages 0 to 49
Parsing pages 50 to 99
Parsing pages 100 to 149
Parsing pages 150 to 199
Parsing pages 200 to 249
Parsing pages 250 to 299
Parsing pages 300 to 349
Parsing pages 350 to 399
Parsing pages 400 to 449
Parsing pages 450 to 499
Parsing pages 500 to 549
Parsing pages 550 to 599
Parsing pages 600 to 649
Parsing pages 650 to 699
Parsing pages 700 to 749
Parsing pages 750 to 799
Parsing pages 800 to 849
Parsing pages 850 to 899
Parsing pages 900 to 949
Parsing pages 950 to 999
Parsing pages 1000 to 1049
Parsing pages 1050 to 1099
Parsing pages 1100 to 1149
Parsing pages 1150 to 1199
Parsing pages 1200 to 1249
Parsing pages 1250 to 1299
Parsing pages 1300 to 1349
Parsing pages 1350 to 1399
Parsing pages 1400 to 1449
Parsing pages 1450 to 1499
Parsing pages 1500 to 1549
Parsing pages 1550 to 1599
Parsing pages 1600 to 1649
Parsing pages 1650 to 1699
Parsing pages 1700 to 1749
Parsing pages 1750 to 1799
Parsing pages 1800 to 1849
Parsing pages 1850 to 1899
Parsing pages 1900 

Parsing pages 14900 to 14949
Parsing pages 14950 to 14999
Parsing pages 15000 to 15049
Parsing pages 15050 to 15099
Parsing pages 15100 to 15149
Parsing pages 15150 to 15199
Parsing pages 15200 to 15249
Parsing pages 15250 to 15299
Parsing pages 15300 to 15349
Parsing pages 15350 to 15399
Parsing pages 15400 to 15449
Parsing pages 15450 to 15499
Parsing pages 15500 to 15549
Parsing pages 15550 to 15599
Parsing pages 15600 to 15649
Parsing pages 15650 to 15699
Parsing pages 15700 to 15749
Parsing pages 15750 to 15799
Parsing pages 15800 to 15849
Parsing pages 15850 to 15899
Parsing pages 15900 to 15949
Parsing pages 15950 to 15999
Parsing pages 16000 to 16049
Parsing pages 16050 to 16099
Parsing pages 16100 to 16149
Parsing pages 16150 to 16199
Parsing pages 16200 to 16249
Parsing pages 16250 to 16299
Parsing pages 16300 to 16349
Parsing pages 16350 to 16399
Parsing pages 16400 to 16449
Parsing pages 16450 to 16499
Parsing pages 16500 to 16549
Parsing pages 16550 to 16599
Parsing pages 

Parsing pages 29050 to 29099
Parsing pages 29100 to 29149
Parsing pages 29150 to 29199
Parsing pages 29200 to 29249
Parsing pages 29250 to 29299
Parsing pages 29300 to 29349
Parsing pages 29350 to 29399
Parsing pages 29400 to 29449
Parsing pages 29450 to 29499
Parsing pages 29500 to 29549
Parsing pages 29550 to 29599
Parsing pages 29600 to 29649
Parsing pages 29650 to 29699
Parsing pages 29700 to 29749
Parsing pages 29750 to 29799
Parsing pages 29800 to 29849
Parsing pages 29850 to 29899
Parsing pages 29900 to 29949
Parsing pages 29950 to 29999
Parsing pages 30000 to 30049
Parsing pages 30050 to 30099
Parsing pages 30100 to 30149
Parsing pages 30150 to 30199
Parsing pages 30200 to 30249
Parsing pages 30250 to 30299
Parsing pages 30300 to 30349
Parsing pages 30350 to 30399
Parsing pages 30400 to 30449
Parsing pages 30450 to 30499
Parsing pages 30500 to 30549
Parsing pages 30550 to 30599
Parsing pages 30600 to 30649
Parsing pages 30650 to 30699
Parsing pages 30700 to 30749
Parsing pages 

Parsing pages 43200 to 43249
Parsing pages 43250 to 43299
Parsing pages 43300 to 43349
Parsing pages 43350 to 43399
Parsing pages 43400 to 43449
Parsing pages 43450 to 43499
Parsing pages 43500 to 43549
Parsing pages 43550 to 43599
Parsing pages 43600 to 43649
Parsing pages 43650 to 43699
Parsing pages 43700 to 43749
Parsing pages 43750 to 43799
Parsing pages 43800 to 43849
Parsing pages 43850 to 43899
Parsing pages 43900 to 43949
Parsing pages 43950 to 43999
Parsing pages 44000 to 44049
Parsing pages 44050 to 44099
Parsing pages 44100 to 44149
Parsing pages 44150 to 44199
Parsing pages 44200 to 44249
Parsing pages 44250 to 44299
Parsing pages 44300 to 44349
Parsing pages 44350 to 44399
Parsing pages 44400 to 44449
Parsing pages 44450 to 44499
Parsing pages 44500 to 44549
Parsing pages 44550 to 44599
Parsing pages 44600 to 44649
Parsing pages 44650 to 44699
Parsing pages 44700 to 44749
Parsing pages 44750 to 44799
Parsing pages 44800 to 44849
Parsing pages 44850 to 44899
Parsing pages 

Parsing pages 57350 to 57399
Parsing pages 57400 to 57449
Parsing pages 57450 to 57499
Parsing pages 57500 to 57549
Parsing pages 57550 to 57599
Parsing pages 57600 to 57649
Parsing pages 57650 to 57699
Parsing pages 57700 to 57749
Parsing pages 57750 to 57799
Parsing pages 57800 to 57849
Parsing pages 57850 to 57899
Parsing pages 57900 to 57949
Parsing pages 57950 to 57999
Parsing pages 58000 to 58049
Parsing pages 58050 to 58099
Parsing pages 58100 to 58149
Parsing pages 58150 to 58199
Parsing pages 58200 to 58249
Parsing pages 58250 to 58299
Parsing pages 58300 to 58349
Parsing pages 58350 to 58399
Parsing pages 58400 to 58449
Parsing pages 58450 to 58499
Parsing pages 58500 to 58549
Parsing pages 58550 to 58599
Parsing pages 58600 to 58649
Parsing pages 58650 to 58699
Parsing pages 58700 to 58749
Parsing pages 58750 to 58799
Parsing pages 58800 to 58849
Parsing pages 58850 to 58899
Parsing pages 58900 to 58949
Parsing pages 58950 to 58999
Parsing pages 59000 to 59049
Parsing pages 

...

Parsing pages 145350 to 145399
Parsing pages 145400 to 145449
Parsing pages 145450 to 145499
Parsing pages 145500 to 145549
Parsing pages 145550 to 145599
Parsing pages 145600 to 145649
Parsing pages 145650 to 145699
Parsing pages 145700 to 145749
Parsing pages 145750 to 145799
Parsing pages 145800 to 145849
Parsing pages 145850 to 145899
Parsing pages 145900 to 145949
Parsing pages 145950 to 145999
Parsing pages 146000 to 146049
Parsing pages 146050 to 146099
Parsing pages 146100 to 146149
Parsing pages 146150 to 146199
Parsing pages 146200 to 146249
Parsing pages 146250 to 146299
Parsing pages 146300 to 146349
Parsing pages 146350 to 146399
Parsing pages 146400 to 146449
Parsing pages 146450 to 146499
Parsing pages 146500 to 146549
Parsing pages 146550 to 146599
Parsing pages 146600 to 146649
Parsing pages 146650 to 146699
Parsing pages 146700 to 146749
Parsing pages 146750 to 146799
Parsing pages 146800 to 146849
Parsing pages 146850 to 146899
Parsing pages 146900 to 146949
Parsing 

Parsing pages 158600 to 158649
Parsing pages 158650 to 158699
Parsing pages 158700 to 158749
Parsing pages 158750 to 158799
Parsing pages 158800 to 158849
Parsing pages 158850 to 158899
Parsing pages 158900 to 158949
Parsing pages 158950 to 158999
Parsing pages 159000 to 159049
Parsing pages 159050 to 159099
Parsing pages 159100 to 159149
Parsing pages 159150 to 159199
Parsing pages 159200 to 159249
Parsing pages 159250 to 159299
Parsing pages 159300 to 159349
Parsing pages 159350 to 159399
Parsing pages 159400 to 159449
Parsing pages 159450 to 159499
Parsing pages 159500 to 159549
Parsing pages 159550 to 159599
Parsing pages 159600 to 159649
Parsing pages 159650 to 159699
Parsing pages 159700 to 159749
Parsing pages 159750 to 159799
Parsing pages 159800 to 159849
Parsing pages 159850 to 159899
Parsing pages 159900 to 159949
Parsing pages 159950 to 159999
Parsing pages 160000 to 160049
Parsing pages 160050 to 160099
Parsing pages 160100 to 160149
Parsing pages 160150 to 160199
Parsing 

Parsing pages 171850 to 171899
Parsing pages 171900 to 171949
Parsing pages 171950 to 171999
Parsing pages 172000 to 172049
Parsing pages 172050 to 172099
Parsing pages 172100 to 172149
Parsing pages 172150 to 172199
Parsing pages 172200 to 172249
Parsing pages 172250 to 172299
Parsing pages 172300 to 172349
Parsing pages 172350 to 172399
Parsing pages 172400 to 172449
Parsing pages 172450 to 172499
Parsing pages 172500 to 172549
Parsing pages 172550 to 172599
Parsing pages 172600 to 172649
Parsing pages 172650 to 172699
Parsing pages 172700 to 172749
Parsing pages 172750 to 172799
Parsing pages 172800 to 172849
Parsing pages 172850 to 172899
Parsing pages 172900 to 172949
Parsing pages 172950 to 172999
Parsing pages 173000 to 173049
Parsing pages 173050 to 173099
Parsing pages 173100 to 173149
Parsing pages 173150 to 173199
Parsing pages 173200 to 173249
Parsing pages 173250 to 173299
Parsing pages 173300 to 173349
Parsing pages 173350 to 173399
Parsing pages 173400 to 173449
Parsing 

Parsing pages 185100 to 185149
Parsing pages 185150 to 185199
Parsing pages 185200 to 185249
Parsing pages 185250 to 185299
Parsing pages 185300 to 185349
Parsing pages 185350 to 185399
Parsing pages 185400 to 185449
Parsing pages 185450 to 185499
Parsing pages 185500 to 185549
Parsing pages 185550 to 185599
Parsing pages 185600 to 185649
Parsing pages 185650 to 185699
Parsing pages 185700 to 185749
Parsing pages 185750 to 185799
Parsing pages 185800 to 185849
Parsing pages 185850 to 185899
Parsing pages 185900 to 185949
Parsing pages 185950 to 185999
Parsing pages 186000 to 186049
Parsing pages 186050 to 186099
Parsing pages 186100 to 186149
Parsing pages 186150 to 186199
Parsing pages 186200 to 186249
Parsing pages 186250 to 186299
Parsing pages 186300 to 186349
Parsing pages 186350 to 186399
Parsing pages 186400 to 186449
Parsing pages 186450 to 186499
Parsing pages 186500 to 186549
Parsing pages 186550 to 186599
Parsing pages 186600 to 186649
Parsing pages 186650 to 186699
Parsing 

Parsing pages 198350 to 198399
Parsing pages 198400 to 198449
Parsing pages 198450 to 198499
Parsing pages 198500 to 198549
Parsing pages 198550 to 198599
Parsing pages 198600 to 198649
Parsing pages 198650 to 198699
Parsing pages 198700 to 198749
Parsing pages 198750 to 198799
Parsing pages 198800 to 198849
Parsing pages 198850 to 198899
Parsing pages 198900 to 198949
Parsing pages 198950 to 198999
Parsing pages 199000 to 199049
Parsing pages 199050 to 199099
Parsing pages 199100 to 199149
Parsing pages 199150 to 199199
Parsing pages 199200 to 199249
Parsing pages 199250 to 199299
Parsing pages 199300 to 199349
Parsing pages 199350 to 199399
Parsing pages 199400 to 199449
Parsing pages 199450 to 199499
Parsing pages 199500 to 199549
Parsing pages 199550 to 199599
Parsing pages 199600 to 199649
Parsing pages 199650 to 199699
Parsing pages 199700 to 199749
Parsing pages 199750 to 199799
Parsing pages 199800 to 199849
Parsing pages 199850 to 199899
Parsing pages 199900 to 199949
Parsing 

In [96]:
test_7 = spotify_parse_missing_ids(z[900000:])

Parsing pages 0 to 49
Parsing pages 50 to 99
Parsing pages 100 to 149
Parsing pages 150 to 199
Parsing pages 200 to 249
Parsing pages 250 to 299
Parsing pages 300 to 349
Parsing pages 350 to 399
Parsing pages 400 to 449
Parsing pages 450 to 499
Parsing pages 500 to 549
Parsing pages 550 to 599
Parsing pages 600 to 649
Parsing pages 650 to 699
Parsing pages 700 to 749
Parsing pages 750 to 799
Parsing pages 800 to 849
Parsing pages 850 to 899
Parsing pages 900 to 949
Parsing pages 950 to 999
Parsing pages 1000 to 1049
Parsing pages 1050 to 1099
Parsing pages 1100 to 1149
Parsing pages 1150 to 1199
Parsing pages 1200 to 1249
Parsing pages 1250 to 1299
Parsing pages 1300 to 1349
Parsing pages 1350 to 1399
Parsing pages 1400 to 1449
Parsing pages 1450 to 1499
Parsing pages 1500 to 1549
Parsing pages 1550 to 1599
Parsing pages 1600 to 1649
Parsing pages 1650 to 1699
Parsing pages 1700 to 1749
Parsing pages 1750 to 1799
Parsing pages 1800 to 1849
Parsing pages 1850 to 1899
Parsing pages 1900 

Parsing pages 14900 to 14949
Parsing pages 14950 to 14999
Parsing pages 15000 to 15049
Parsing pages 15050 to 15099
Parsing pages 15100 to 15149
Parsing pages 15150 to 15199
Parsing pages 15200 to 15249
Parsing pages 15250 to 15299
Parsing pages 15300 to 15349
Parsing pages 15350 to 15399
Parsing pages 15400 to 15449
Parsing pages 15450 to 15499
Parsing pages 15500 to 15549
Parsing pages 15550 to 15599
Parsing pages 15600 to 15649
Parsing pages 15650 to 15699
Parsing pages 15700 to 15749
Parsing pages 15750 to 15799
Parsing pages 15800 to 15849
Parsing pages 15850 to 15899
Parsing pages 15900 to 15949
Parsing pages 15950 to 15999
Parsing pages 16000 to 16049
Parsing pages 16050 to 16099
Parsing pages 16100 to 16149
Parsing pages 16150 to 16199
Parsing pages 16200 to 16249
Parsing pages 16250 to 16299
Parsing pages 16300 to 16349
Parsing pages 16350 to 16399
Parsing pages 16400 to 16449
Parsing pages 16450 to 16499
Parsing pages 16500 to 16549
Parsing pages 16550 to 16599
Parsing pages 

Parsing pages 29050 to 29099
Parsing pages 29100 to 29149
Parsing pages 29150 to 29199
Parsing pages 29200 to 29249
Parsing pages 29250 to 29299
Parsing pages 29300 to 29349
Parsing pages 29350 to 29399
Parsing pages 29400 to 29449
Parsing pages 29450 to 29499
Parsing pages 29500 to 29549
Parsing pages 29550 to 29599
Parsing pages 29600 to 29649
Parsing pages 29650 to 29699
Parsing pages 29700 to 29749
Parsing pages 29750 to 29799
Parsing pages 29800 to 29849
Parsing pages 29850 to 29899
Parsing pages 29900 to 29949
Parsing pages 29950 to 29999
Parsing pages 30000 to 30049
Parsing pages 30050 to 30099
Parsing pages 30100 to 30149
Parsing pages 30150 to 30199
Parsing pages 30200 to 30249
Parsing pages 30250 to 30299
Parsing pages 30300 to 30349
Parsing pages 30350 to 30399
Parsing pages 30400 to 30449
Parsing pages 30450 to 30499
Parsing pages 30500 to 30549
Parsing pages 30550 to 30599
Parsing pages 30600 to 30649
Parsing pages 30650 to 30699
Parsing pages 30700 to 30749
Parsing pages 

Parsing pages 43200 to 43249
Parsing pages 43250 to 43299
Parsing pages 43300 to 43349
Parsing pages 43350 to 43399
Parsing pages 43400 to 43449
Parsing pages 43450 to 43499
Parsing pages 43500 to 43549
Parsing pages 43550 to 43599
Parsing pages 43600 to 43649
Parsing pages 43650 to 43699
Parsing pages 43700 to 43749
Parsing pages 43750 to 43799
Parsing pages 43800 to 43849
Parsing pages 43850 to 43899
Parsing pages 43900 to 43949
Parsing pages 43950 to 43999
Parsing pages 44000 to 44049
Parsing pages 44050 to 44099
Parsing pages 44100 to 44149
Parsing pages 44150 to 44199
Parsing pages 44200 to 44249
Parsing pages 44250 to 44299
Parsing pages 44300 to 44349
Parsing pages 44350 to 44399
Parsing pages 44400 to 44449
Parsing pages 44450 to 44499
Parsing pages 44500 to 44549
Parsing pages 44550 to 44599
Parsing pages 44600 to 44649
Parsing pages 44650 to 44699
Parsing pages 44700 to 44749
Parsing pages 44750 to 44799
Parsing pages 44800 to 44849
Parsing pages 44850 to 44899
Parsing pages 

Parsing pages 57350 to 57399
Parsing pages 57400 to 57449
Parsing pages 57450 to 57499
Parsing pages 57500 to 57549
Parsing pages 57550 to 57599
Parsing pages 57600 to 57649
Parsing pages 57650 to 57699
Parsing pages 57700 to 57749
Parsing pages 57750 to 57799
Parsing pages 57800 to 57849
Parsing pages 57850 to 57899
Parsing pages 57900 to 57949
Parsing pages 57950 to 57999
Parsing pages 58000 to 58049
Parsing pages 58050 to 58099
Parsing pages 58100 to 58149
Parsing pages 58150 to 58199
Parsing pages 58200 to 58249
Parsing pages 58250 to 58299
Parsing pages 58300 to 58349
Parsing pages 58350 to 58399
Parsing pages 58400 to 58449
Parsing pages 58450 to 58499
Parsing pages 58500 to 58549
Parsing pages 58550 to 58599
Parsing pages 58600 to 58649
Parsing pages 58650 to 58699
Parsing pages 58700 to 58749
Parsing pages 58750 to 58799
Parsing pages 58800 to 58849
Parsing pages 58850 to 58899
Parsing pages 58900 to 58949
Parsing pages 58950 to 58999
Parsing pages 59000 to 59049
Parsing pages 

...

Parsing pages 126300 to 126349
Parsing pages 126350 to 126399
Parsing pages 126400 to 126449
Parsing pages 126450 to 126499
Parsing pages 126500 to 126549
Parsing pages 126550 to 126599
Parsing pages 126600 to 126649
Parsing pages 126650 to 126699
Parsing pages 126700 to 126749
Parsing pages 126750 to 126799
Parsing pages 126800 to 126849
Parsing pages 126850 to 126899
Parsing pages 126900 to 126949
Parsing pages 126950 to 126999
Parsing pages 127000 to 127049
Parsing pages 127050 to 127099
Parsing pages 127100 to 127149
Parsing pages 127150 to 127199
Parsing pages 127200 to 127249
Parsing pages 127250 to 127299
Parsing pages 127300 to 127349
Parsing pages 127350 to 127399
Parsing pages 127400 to 127449
Parsing pages 127450 to 127499
Parsing pages 127500 to 127549
Parsing pages 127550 to 127599
Parsing pages 127600 to 127649
Parsing pages 127650 to 127699
Parsing pages 127700 to 127749
Parsing pages 127750 to 127799
Parsing pages 127800 to 127849
Parsing pages 127850 to 127899
Parsing 

Parsing pages 139550 to 139599
Parsing pages 139600 to 139649
Parsing pages 139650 to 139699
Parsing pages 139700 to 139749
Parsing pages 139750 to 139799
Parsing pages 139800 to 139849
Parsing pages 139850 to 139899
Parsing pages 139900 to 139949
Parsing pages 139950 to 139999
Parsing pages 140000 to 140049
Parsing pages 140050 to 140099
Parsing pages 140100 to 140149
Parsing pages 140150 to 140199
Parsing pages 140200 to 140249
Parsing pages 140250 to 140299
Parsing pages 140300 to 140349
Parsing pages 140350 to 140399
Parsing pages 140400 to 140449
Parsing pages 140450 to 140499
Parsing pages 140500 to 140549
Parsing pages 140550 to 140599
Parsing pages 140600 to 140649
Parsing pages 140650 to 140699
Parsing pages 140700 to 140749
Parsing pages 140750 to 140799
Parsing pages 140800 to 140849
Parsing pages 140850 to 140899
Parsing pages 140900 to 140949
Parsing pages 140950 to 140999
Parsing pages 141000 to 141049
Parsing pages 141050 to 141099
Parsing pages 141100 to 141149
Parsing 

Parsing pages 152800 to 152849
Parsing pages 152850 to 152899
Parsing pages 152900 to 152949
Parsing pages 152950 to 152999
Parsing pages 153000 to 153049
Parsing pages 153050 to 153099
Parsing pages 153100 to 153149
Parsing pages 153150 to 153199
Parsing pages 153200 to 153249
Parsing pages 153250 to 153299
Parsing pages 153300 to 153349
Parsing pages 153350 to 153399
Parsing pages 153400 to 153449
Parsing pages 153450 to 153499
Parsing pages 153500 to 153549
Parsing pages 153550 to 153599
Parsing pages 153600 to 153649
Parsing pages 153650 to 153699
Parsing pages 153700 to 153749
Parsing pages 153750 to 153799
Parsing pages 153800 to 153849
Parsing pages 153850 to 153899
Parsing pages 153900 to 153949
Parsing pages 153950 to 153999
Parsing pages 154000 to 154049
Parsing pages 154050 to 154099
Parsing pages 154100 to 154149
Parsing pages 154150 to 154199
Parsing pages 154200 to 154249
Parsing pages 154250 to 154299
Parsing pages 154300 to 154349
Parsing pages 154350 to 154399
Parsing 

Parsing pages 166050 to 166099
Parsing pages 166100 to 166149
Parsing pages 166150 to 166199
Parsing pages 166200 to 166249
Parsing pages 166250 to 166299
Parsing pages 166300 to 166349
Parsing pages 166350 to 166399
Parsing pages 166400 to 166449
Parsing pages 166450 to 166499
Parsing pages 166500 to 166549
Parsing pages 166550 to 166599
Parsing pages 166600 to 166649
Parsing pages 166650 to 166699
Parsing pages 166700 to 166749
Parsing pages 166750 to 166799
Parsing pages 166800 to 166849
Parsing pages 166850 to 166899
Parsing pages 166900 to 166949
Parsing pages 166950 to 166999
Parsing pages 167000 to 167049
Parsing pages 167050 to 167099
Parsing pages 167100 to 167149
Parsing pages 167150 to 167199
Parsing pages 167200 to 167249
Parsing pages 167250 to 167299
Parsing pages 167300 to 167349
Parsing pages 167350 to 167399
Parsing pages 167400 to 167449
Parsing pages 167450 to 167499
Parsing pages 167500 to 167549
Parsing pages 167550 to 167599
Parsing pages 167600 to 167649
Parsing 

Parsing pages 179300 to 179349
Parsing pages 179350 to 179399
Parsing pages 179400 to 179449
Parsing pages 179450 to 179499
Parsing pages 179500 to 179549
Parsing pages 179550 to 179599
Parsing pages 179600 to 179649
Parsing pages 179650 to 179699
Parsing pages 179700 to 179749
Parsing pages 179750 to 179799
Parsing pages 179800 to 179849
Parsing pages 179850 to 179899
Parsing pages 179900 to 179949
Parsing pages 179950 to 179999
Parsing pages 180000 to 180049
Parsing pages 180050 to 180099
Parsing pages 180100 to 180149
Parsing pages 180150 to 180199
Parsing pages 180200 to 180249
Parsing pages 180250 to 180299
Parsing pages 180300 to 180349
Parsing pages 180350 to 180399
Parsing pages 180400 to 180449
Parsing pages 180450 to 180499
Parsing pages 180500 to 180549
Parsing pages 180550 to 180599
Parsing pages 180600 to 180649
Parsing pages 180650 to 180699
Parsing pages 180700 to 180749
Parsing pages 180750 to 180799
Parsing pages 180800 to 180849
Parsing pages 180850 to 180899
Parsing 

In [102]:
to_concat = [test_1, test_2, test_3, test_4, test_5, test_6, test_7, df_master]
df_master = pd.concat(to_concat)

In [103]:
df_master

,album,artist,disc_number,duration,id,popularity,track,track_number
0,Sleep Lullaby Special,Agni Bjarki,1,162519,4hi5AvFqHQufzYIwputosy,10,Tomorrow's song,38
1,3 Years of Vamos Music Selected by Rober Gaez,Ivan Pica,1,369836,6F4I6j5JWaNFmJFjzMfVAF,0,Acidman - Juanito Remix,11
2,My Prescription,Bobby Womack,1,157960,2csPJDHt9KPsXiHMV3nLhG,15,Everyone's Gone To The Moon,4
3,Jam In The Van - Aaron Woody Wood,Aaron Woody Wood,1,298762,2SnWO5e0WCWoGH1n7Sn9ZB,0,"Big City - Live from Asheville, NC 2016",4
4,The Fifth (Deluxe),Dizzee Rascal,1,219240,3kua24ltWzq87a42rDekNy,23,H-Town,11
5,Illasophic Vol. 1.5,Myka 9,1,239885,1ZiGzi0KsSeDthO5ghupQB,1,"Adapt & Overcome (feat. Adlib, Nairb, Samson, ...",9
6,Collectors Series-Garnett Silk,Garnett Silk,1,243666,1SAK6BEuK8KdyX2V90a94y,14,Fussing & Fighting,8
7,Code of Conduct,Funker Vogt,1,312986,4WINnfpUAb2z3Y6Z0i7c29,19,Kämpfer,12
8,Poison,Karim,1,312293,3gbj24M6KTVZe5mpTfRT8O,0,Poison - Origial Outwork Mix,1
9,Back to Me,Dapayk solo,1,549000,7yf9YP1oYP9RYByzp7Q9mh,0,Back to Me,1


In [46]:
df_master.columns

Index(['release', 'artist', 'disc_number', 'duration', 'id', 'popularity',
       'track', 'track_number'],
      dtype='object')

In [54]:
z_1['tracks'][43]['album']['name']

'3sixty (Can U Dig It)'

In [71]:
concat_again = []
for i in concat_these:
    concat_again.append(pd.concat(i))
df_missing_master = pd.concat(concat_again)

In [73]:
df_missing_master = df_missing_master.reset_index(drop = True)

In [ ]:
df_missing_master.to_csv("missing_spotify_audio_features.csv")

In [28]:
spotify.audio_features('44ip8N5VHZP9EXwIcgfGUn')

[{'acousticness': 0.00169,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/44ip8N5VHZP9EXwIcgfGUn',
  'danceability': 0.728,
  'duration_ms': 454217,
  'energy': 0.462,
  'id': '44ip8N5VHZP9EXwIcgfGUn',
  'instrumentalness': 0.878,
  'key': 10,
  'liveness': 0.0704,
  'loudness': -6.059,
  'mode': 0,
  'speechiness': 0.0493,
  'tempo': 136.002,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/44ip8N5VHZP9EXwIcgfGUn',
  'type': 'audio_features',
  'uri': 'spotify:track:44ip8N5VHZP9EXwIcgfGUn',
  'valence': 0.191}]

In [27]:
spotify.audio_features('0HMRs0596MY9nBE7MkwfbL')

[{'acousticness': 0.736,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0HMRs0596MY9nBE7MkwfbL',
  'danceability': 0.686,
  'duration_ms': 553831,
  'energy': 0.425,
  'id': '0HMRs0596MY9nBE7MkwfbL',
  'instrumentalness': 0.952,
  'key': 1,
  'liveness': 0.0953,
  'loudness': -19.218,
  'mode': 1,
  'speechiness': 0.0896,
  'tempo': 125.98,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/0HMRs0596MY9nBE7MkwfbL',
  'type': 'audio_features',
  'uri': 'spotify:track:0HMRs0596MY9nBE7MkwfbL',
  'valence': 0.256}]

In [29]:
princeofdenmark = spotify.album('0YbqP2YSSgUwnbY8YnyCT5') # prince of denmark

In [32]:
princeofdenmark['tracks']['items']

[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qxYMvz3XRdQROq9Wh8EH6'},
    'href': 'https://api.spotify.com/v1/artists/6qxYMvz3XRdQROq9Wh8EH6',
    'id': '6qxYMvz3XRdQROq9Wh8EH6',
    'name': 'Prince of Denmark',
    'type': 'artist',
    'uri': 'spotify:artist:6qxYMvz3XRdQROq9Wh8EH6'}],
  'available_markets': [],
  'disc_number': 1,
  'duration_ms': 395900,
  'explicit': False,
  'external_urls': {'spotify': 'https://open.spotify.com/track/6vApVe3LCHA9MU19raHQRm'},
  'href': 'https://api.spotify.com/v1/tracks/6vApVe3LCHA9MU19raHQRm',
  'id': '6vApVe3LCHA9MU19raHQRm',
  'name': 'Intro',
  'preview_url': None,
  'track_number': 1,
  'type': 'track',
  'uri': 'spotify:track:6vApVe3LCHA9MU19raHQRm'},
 {'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qxYMvz3XRdQROq9Wh8EH6'},
    'href': 'https://api.spotify.com/v1/artists/6qxYMvz3XRdQROq9Wh8EH6',
    'id': '6qxYMvz3XRdQROq9Wh8EH6',
    'name': 'Prince of Denmark',
    'type': 'art